## Installing Google GenAI Library



In [11]:
!pip install -Uq "google-genai==1.7.0"

## Import necessary libraries
1. Importing Libraries
The code begins by importing necessary libraries:

google.genai for interacting with Google's Generative AI services.

IPython.display for displaying Markdown content.

enum for defining enumeration types.

kaggle_secrets for managing API keys securely.

In [12]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

genai.__version__

from kaggle_secrets import UserSecretsClient
import enum

client = genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY"))

In [13]:
!wget -nv -O Generative-AI-in-the-Newsroom.pdf "https://fleishmanhillard.com/wp-content/uploads/2024/06/Generative-AI-in-the-Newsroom.pdf"
document_file = client.files.upload(file='Generative-AI-in-the-Newsroom.pdf')

2025-04-02 18:01:01 URL:https://fleishmanhillard.com/wp-content/uploads/2024/06/Generative-AI-in-the-Newsroom.pdf [9265681/9265681] -> "Generative-AI-in-the-Newsroom.pdf" [1]


### Define the prompt for evaluating summaries

In [14]:


# Constants
SUMMARY_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by AI models.
We will provide you with the user input and an AI-generated responses.
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.

# Evaluation
## Metric Definition
You will be assessing summarization quality, which measures the overall ability to summarize text. Pay special attention to length constraints, such as in X words or in Y sentences. The instruction for performing a summarization task and the context to be summarized are provided in the user prompt. The response should be shorter than the text in the context. The response should not contain information that is not present in the context.

## Criteria
Instruction following: The response demonstrates a clear understanding of the summarization task instructions, satisfying all of the instruction's requirements.
Groundedness: The response contains information included only in the context. The response does not reference any outside information.
Conciseness: The response summarizes the relevant details in the original text without a significant loss in key information without being too verbose or terse.
Fluency: The response is well-organized and easy to read.

## Rating Rubric
5: (Very good). The summary follows instructions, is grounded, is concise, and fluent.
4: (Good). The summary follows instructions, is grounded, concise, and fluent.
3: (Ok). The summary mostly follows instructions, is grounded, but is not very concise and is not fluent.
2: (Bad). The summary is grounded, but does not follow the instructions.
1: (Very bad). The summary is not grounded.

## Evaluation Steps
STEP 1: Assess the response in aspects of instruction following, groundedness, conciseness, and verbosity according to the criteria.
STEP 2: Score based on the rubric.

# User Inputs and AI-generated Response
## User Inputs

### Prompt
{prompt}

## AI-generated Response
{response}
"""

# Setup
def setup_api_client(api_key):
    """Sets up the API client."""
    client = genai.Client(api_key=api_key)
    return client

# Evaluation
class SummaryRating(enum.Enum):
    VERY_GOOD = '5'
    GOOD = '4'
    OK = '3'
    BAD = '2'
    VERY_BAD = '1'

def eval_summary(prompt, ai_response, client):
    """Evaluates the generated summary against the prompt used."""
    try:
        chat = client.chats.create(model='gemini-2.0-flash')
        
        # Generate the full text response.
        response = chat.send_message(
            message=SUMMARY_PROMPT.format(prompt=prompt, response=ai_response)
        )
        verbose_eval = response.text
        
        # Coerce into the desired structure.
        structured_output_config = types.GenerateContentConfig(
            response_mime_type="text/x.enum",
            response_schema=SummaryRating,
        )
        response = chat.send_message(
            message="Convert the final score.",
            config=structured_output_config,
        )
        structured_eval = response.parsed
        
        return verbose_eval, structured_eval
    except genai.errors.APIError as e:
        print(f"API Error: {e}")
        return None, None

# Summarization
def summarise_doc(request, document_file, client):
    """Executes the request on the uploaded document."""
    # Set the temperature low to stabilise the output.
    config = types.GenerateContentConfig(temperature=0.0)
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        config=config,
        contents=[request, document_file],
    )
    
    return response.text




In [15]:
# Main Execution
if __name__ == "__main__":
    api_key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    client = setup_api_client(api_key)
    
    request = 'Are content creation and data analysis utilized to the same degree?'
    document_file = client.files.upload(file='Generative-AI-in-the-Newsroom.pdf')
    
    summary = summarise_doc(request, document_file, client)
    display(Markdown(summary))
    
    text_eval, struct_eval = eval_summary([request, document_file], summary, client)
    display(Markdown(text_eval))
    print(struct_eval)

Based on the document provided, here's an analysis of how content creation and data analysis are utilized in newsrooms, according to the study:

**Key Takeaways from the Document:**

*   **Data Analysis is Currently More Prevalent:** The document explicitly states, "While many aspects of content creation and data analysis overlap, data analysis is currently a more prevalent use case of artificial intelligence due to its versatility."
*   **Content Creation is Used, but with Caution:** The document indicates that GenAI is used for content creation, but with more skepticism and regulation. Concerns about bias, offensive content, and fabricated information are key factors.
*   **Data Analysis Applications:** The document highlights specific data analysis applications, such as:
    *   Backend automation (tagging, fact-checking, sorting).
    *   Audience profiling (monitoring trends, social media sentiment).
    *   Generating financial and news reports.
    *   Optimizing paywalls.
    *   Summarizing articles.
    *   Delivering personalized advertisements and recommendations.
*   **Content Creation Applications:** The document mentions content creation applications, such as:
    *   Editorial assistance (drafting articles, generating story narratives, ensuring grammatical accuracy).
    *   Multimedia production (media reformatting, transcription, translation, and generation of new content like images and videos).
    *   Automated reporting (generating news stories from data inputs).
    *   Content personalization (tailoring content to individual readers' preferences).
*   **Ethical Concerns:** The document emphasizes ethical concerns related to content creation, including misinformation, bias, and erosion of trust.
*   **Early Adopters vs. Critics:** The document highlights that while some news organizations are embracing GenAI for content creation and data analysis, others are more cautious due to concerns about quality, accuracy, and ethical implications.

**In Summary:**

While both content creation and data analysis are leveraging GenAI in newsrooms, data analysis appears to be more widely adopted and integrated at this stage. This is likely due to its versatility in automating backend tasks, audience profiling, and generating reports. Content creation is being explored, but with more caution and regulation due to concerns about quality, accuracy, and ethical implications.

STEP 1: The response summarizes the document and answers the prompt question. It extracts the main points from the document and organizes them in a coherent way. The response is fluent and easy to read.
STEP 2: Score 5.

SummaryRating.VERY_GOOD
